In [1]:
import ROOT
ROOT.gROOT.SetBatch(True)
ROOT.gSystem.Load("libFWCoreFWLite.so");
ROOT.gSystem.Load("libDataFormatsFWLite.so");
ROOT.FWLiteEnabler.enable()
from DataFormats.FWLite import Handle, Events


Warning in <TInterpreter::ReadRootmapFile>: class  edm::Wrapper<std::vector<bool> > found in libDataFormatsWrappedStdDictionaries.so  is already in libOSUT3AnalysisAnaTools.so 
Warning in <TInterpreter::ReadRootmapFile>: class  edm::Wrapper<vector<bool> > found in libDataFormatsWrappedStdDictionaries.so  is already in libOSUT3AnalysisAnaTools.so 


Welcome to JupyROOT 6.24/07


In [2]:
isoTracks, isoTracksLabel = Handle("std::vector<pat::IsolatedTrack>"), "isolatedTracks"
muons, muonLabel = Handle("std::vector<pat::Muon>"), "slimmedMuons"
tracks, tracksLabel = Handle("std::vector<reco::Track>"), "generalTracks"


In [ ]:
events = Events('/store/user/mcarrigan/NeutrinoGun_step3_v2/hist_10.root')

h_2022 = ROOT.TH1F("h_2022", "Pt of General Tracks 2022", 200, 0, 100)
h_2017 = ROOT.TH1F("h_2017", "Pt of General Tracks 2017", 200, 0, 100)

In [ ]:
evtCount = 0
for iev,event in enumerate(events):
    #if iev >= 10: break
    event.getByLabel(tracksLabel, tracks)
    #print("\nEvent %d: run %6d, lumi %4d, event %12d" % (iev,event.eventAuxiliary().run(), event.eventAuxiliary().luminosityBlock(),event.eventAuxiliary().event()))
    print("Event {0}, total tracks {1}".format(iev, len(tracks.product())))
    for i,track in enumerate(tracks.product()):
        #print('\t', i, track.pt())
        if track.pt() > 55: evtCount+=1
print("Number of tracks passing pt cut {}".format(evtCount))

In [ ]:
events = Events('root://cmsxrootd.fnal.gov://store/mc/Run3Summer22DRPremix/SingleNeutrino_E-10_gun/AODSIM/SNB_124X_mcRun3_2022_realistic_v12-v2/70000/09ba7969-5522-42bb-bc5a-61543ace1af3.root')
eventsMini = Events('root://cmsxrootd.fnal.gov://store/mc/Run3Summer22MiniAODv3/SingleNeutrino_E-10_gun/MINIAODSIM/SNB_124X_mcRun3_2022_realistic_v12-v2/70000/2e72569a-ed24-48fb-b632-0f4fdd3dc4e5.root')

In [ ]:
evtCount = 0
for iev,event in enumerate(events):
    #if iev >= 10: break
    if iev%1000==0: print("Working on event", iev)
    event.getByLabel(tracksLabel, tracks)
    #print("Event {0}, total tracks {1}".format(iev, len(tracks.product())))
    for i,track in enumerate(tracks.product()):
        #print('\t', i, track.pt())
        if track.pt() > 55: evtCount+=1
print("Number of tracks passing pt cut {}".format(evtCount))

In [3]:
m_pi = 1.570796
def inGap(track):
    TOB = (abs(track.dz()) < 0.5 and abs(m_pi - track.theta()) < 1.0e-3)
    ECAL = (abs(track.eta()) >= 1.42 and abs(track.eta()) <= 1.65)
    DT = abs(track.eta()) >= 0.15 and abs(track.eta()) <= 0.35
    CSC = abs(track.eta()) >= 1.55 and abs(track.eta()) <= 1.85
    inGap = TOB or ECAL or DT or CSC
    return inGap

In [4]:
evtCount = [0,0,0,0,0,0,0,0]
eventsMini = Events('root://cmsxrootd.fnal.gov://store/mc/Run3Summer22MiniAODv3/SingleNeutrino_E-10_gun/MINIAODSIM/SNB_124X_mcRun3_2022_realistic_v12-v2/70000/2e72569a-ed24-48fb-b632-0f4fdd3dc4e5.root')
#eventsMini = Events('/home/mcarrigan/scratch0/disTracksML/CMSSW_9_4_9/src/DisappTrksML/TreeMaker/test/step1_PAT_test.root')
for iev,event in enumerate(eventsMini):
    if iev >= 1000: break
    if iev%10000==0: print("Working on event", iev)
    event.getByLabel(isoTracksLabel, isoTracks)
    #print("Event {0}, total tracks {1}".format(iev, len(tracks.product())))
    for i,track in enumerate(isoTracks.product()):
        evtCount[0] += 1
        #print('\t', i, track.pt())
        if not abs(track.eta()) < 2.1: continue
        evtCount[1] += 1
        if not track.pt() > 25: continue
        evtCount[2] += 1
        if not track.pt() > 55: continue
        evtCount[3] += 1
        if inGap(track): continue
        evtCount[4] += 1
        if not track.hitPattern().numberOfValidPixelHits() >=4: continue
        evtCount[5] += 1
        if not track.hitPattern().numberOfValidHits() >= 4: continue
        evtCount[6] += 1
print("Number of tracks passing cuts {}".format(evtCount))

Working on event 0
Number of tracks passing cuts [1574, 800, 20, 12, 5, 1, 1, 0]


In [ ]:
Number of tracks passing cuts [4341, 3021, 878, 193, 132, 43, 43, 0]


In [ ]:
for iev,event in enumerate(events2017):
    event.getByLabel(tracksLabel, tracks)
    print("Event {0}, total tracks {1}".format(iev, len(tracks.product())))
    for i,track in enumerate(tracks.product()):
        #print('\t', i, track.pt())
        h_2017.Fill(track.pt())

In [ ]:
c1 = ROOT.TCanvas("c1", "c1, 800, 800")
c1.cd()

In [ ]:
h_2022.Draw()
h_2017.SetLineColor(2)
h_2017.Draw("same")
c1.SetLogy()
c1.SetLogx()
c1.Draw()